In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./2.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Make No'] = input_['Make No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Make'].tolist()

总数量：4922


['Acura',
 'Chevrolet',
 'Ford',
 'GMC',
 'Jeep',
 'Kenworth',
 'Kia',
 'Ram',
 'Subaru',
 'Volkswagen',
 'Acura',
 'American Coach',
 'Audi',
 'Autocar LLC.',
 'BMW',
 'Buick',
 'Cadillac',
 'Chevrolet',
 'Chrysler',
 'Coachmen',
 'Dodge',
 'Entegra Coach',
 'Fleetwood',
 'Ford',
 'Foretravel Motorhome',
 'Freightliner',
 'Genesis',
 'GMC',
 'Hino',
 'Holiday Rambler',
 'Honda',
 'Hyundai',
 'IC Corporation',
 'Infiniti',
 'International',
 'Jaguar',
 'Jayco',
 'Jeep',
 'Kenworth',
 'Kia',
 'Land Rover',
 'Lexus',
 'Lincoln',
 'Mack',
 'Mazda',
 'Mercedes-Benz',
 'MG',
 'Mini',
 'Mitsubishi',
 'Newmar',
 'Nexus',
 'Nissan',
 'Peterbilt',
 'Porsche',
 'Ram',
 'Renegade',
 'Spartan Motors',
 'Subaru',
 'Temsa Bus',
 'Tesla',
 'Thomas',
 'Thor Motor Coach',
 'Tiffin',
 'Toyota',
 'Volkswagen',
 'Volvo',
 'Western Star',
 'Winnebago',
 'Acura',
 'Alfa Romeo',
 'American Coach',
 'Audi',
 'Autocar LLC.',
 'Bentley',
 'Blue Bird',
 'BMW',
 'Buick',
 'Cadillac',
 'Capacity Of Texas',
 'Chevr

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'cssClass': 'form-control',
        'dropdownType': '2',
        'selectedValue': '',
        'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                              'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                              'ModelID': 0,
                              'SkinID': '1',
                              'Year': f'''{input_.loc[a, 'Year']}'''}}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                             data=json.dumps(data),
                             headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                             proxies=get_proxy(),
                             timeout=(10, 10))
        
        if resp.status_code == 200:
            break
    except:
        continue

resp

1
2
3
4
5
6
7
8
9
10


<Response [200]>

In [6]:
json_ = resp.json()

json_

{'d': '<select id="modelDropdown" name="modelDropdown" onchange="return onDropdownChanged(3, modelDropdown.selectedIndex == 0, \'parttypeDropdownElement\');" class="form-control">\n<option value="0" selected="1" >{0}</option>\n<option value="742"  >Integra</option>\n<option value="744"  >MDX</option>\n</select>'}

In [7]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['d'], 'lxml')
html = etree.HTML(str(soup))

with open('./model_1.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <select class="form-control" id="modelDropdown" name="modelDropdown" onchange="return onDropdownChanged(3, modelDropdown.selectedIndex == 0, 'parttypeDropdownElement');">
   <option selected="1" value="0">
    {0}
   </option>
   <option value="742">
    Integra
   </option>
   <option value="744">
    MDX
   </option>
  </select>
 </body>
</html>



In [8]:
list_model_code = html.xpath('//select[@id="modelDropdown"]/option/@value')[1:]

list_model_code

['742', '744']

In [9]:
list_model = html.xpath('//select[@id="modelDropdown"]/option/text()')[1:]

list_model

['Integra', 'MDX']

In [10]:
len(list_model) == 0

False

In [11]:
df_temp = pd.DataFrame({'Year': input_.loc[a, 'Year'],
                        'Make No': input_.loc[a, 'Make No'],
                        'Make': input_.loc[a, 'Make'],
                        'Make Code': input_.loc[a, 'Make Code'],
                        'Model No': [i+1 for i in range(len(list_model))],
                        'Model': list_model,
                        'Model Code': list_model_code})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Year,Make No,Make,Make Code,Model No,Model,Model Code
0,2025,1,Acura,58,1,Integra,742
1,2025,1,Acura,58,2,MDX,744


In [12]:
crawler_status = 'ok'

crawler_status

'ok'

In [13]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Year': input_.loc[a, 'Year'],
                             'Make No': input_.loc[a, 'Make No'],
                             'Make': input_.loc[a, 'Make'],
                             'Make Code': input_.loc[a, 'Make Code']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [14]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Year', 'Make No', 'Model No'],
                                                ascending=[False, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_model_1.xlsx', index=False)

output_correct

,Year,Make No,Make,Make Code,Model No,Model,Model Code
0,2025,1,Acura,58,1,Integra,742
1,2025,1,Acura,58,2,MDX,744


In [15]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Year', 'Make No'],
                                            ascending=[False, True],
                                            ignore_index=True)
    output_error.to_excel('./test_model_1_error.xlsx', index=False)

output_error

""
